# 02: All Contract Types — The Complete Reference

KindScript enforces 6 types of architectural checks, organized into three categories:

**Dependency contracts:**

| Contract | Error Code | What it catches |
|----------|------------|----------------|
| `noDependency` | KS70001 | Forbidden imports between layers |
| `mustImplement` | KS70002 | Missing interface implementations |
| `purity` | KS70003 | I/O imports in pure layers |
| `noCycles` | KS70004 | Circular dependencies between layers |

**Filesystem contracts** (declared via `filesystem: { ... }` in ConstraintConfig):

| Contract | Error Code | What it catches |
|----------|------------|----------------|
| `filesystem.exists` | KS70010 | Member directories that don't exist on disk |
| `filesystem.mirrors` | KS70005 | Missing counterpart files (e.g., tests) |

This notebook demonstrates each one with a **violation** and a **fix**.

## Setup

In [ ]:
import { ksc, copyFixture, writeFile, cleanup } from './lib.ts';
console.log("Setup complete.");

---

## A. `noDependency` — Forbidden Imports (KS70001)

The most common contract. Forbids imports from one layer to another.

**Use case:** In Clean Architecture, the domain layer must not import from infrastructure.

```typescript
noDependency: [
  ["domain", "infrastructure"],  // domain cannot import from infrastructure
]
```

In [ ]:
const demo1 = copyFixture("no-dependency");

writeFile(demo1, "src/context.k.ts", `
import type { Kind, ConstraintConfig, InstanceConfig } from 'kindscript';

export type DomainLayer = Kind<"DomainLayer">;
export type InfrastructureLayer = Kind<"InfrastructureLayer">;

export type CleanContext = Kind<"CleanContext", {
  domain: DomainLayer;
  infrastructure: InfrastructureLayer;
}, {
  noDependency: [["domain", "infrastructure"]];
}>;

export const app = {
  domain: {},
  infrastructure: {},
} satisfies InstanceConfig<CleanContext>;
`);

console.log("=== Violation: domain imports from infrastructure ===");
await ksc("check", demo1);

In [ ]:
// Fix: domain defines its own interface, no infrastructure import
writeFile(demo1, "src/domain/service.ts", `
export interface DataStore {
  query(sql: string): string[];
}

export class DomainService {
  constructor(private store: DataStore) {}
  getAll(): string[] {
    return this.store.query('SELECT * FROM entities');
  }
}
`);

console.log("=== Fixed: domain uses its own interface ===");
await ksc("check", demo1);

cleanup(demo1);

---

## B. `purity` — No I/O in Pure Layers (KS70003)

Ensures a layer has no side effects — no `fs`, `http`, `net`, `child_process`, or any of Node's ~50 built-in I/O modules.

**Use case:** Domain logic should be pure. If it needs to read a file, it receives the data through a port.

Purity is declared as an **intrinsic constraint** on the leaf Kind type:

```typescript
type DomainLayer = Kind<"DomainLayer", {}, { pure: true }>;
```

When a composite Kind contains a member of this type, purity is automatically enforced.

In [ ]:
const demo2 = copyFixture("purity");

writeFile(demo2, "src/context.k.ts", `
import type { Kind, ConstraintConfig, InstanceConfig } from 'kindscript';

export type DomainLayer = Kind<"DomainLayer", {}, { pure: true }>;

export type AppContext = Kind<"AppContext", {
  domain: DomainLayer;
}>;

export const app = {
  domain: {},
} satisfies InstanceConfig<AppContext>;
`);

console.log("=== Violation: domain imports 'fs' ===");
await ksc("check", demo2);

In [ ]:
// Fix: inject data instead of reading directly
writeFile(demo2, "src/domain/service.ts", `
export interface DataReader {
  read(path: string): string;
}

export class DomainService {
  constructor(private reader: DataReader) {}
  readData(): string {
    return this.reader.read('/tmp/data.txt');
  }
}
`);

console.log("=== Fixed: domain uses injected reader ===");
await ksc("check", demo2);

cleanup(demo2);

---

## C. `mustImplement` — Every Port Needs an Adapter (KS70002)

Ensures every exported interface in one layer has a class that `implements` it in another.

**Use case:** In Hexagonal Architecture, every port interface needs an adapter.

```typescript
mustImplement: [["ports", "adapters"]]
```

In [ ]:
const demo3 = copyFixture("must-implement");

writeFile(demo3, "src/context.k.ts", `
import type { Kind, ConstraintConfig, InstanceConfig } from 'kindscript';

export type PortsLayer = Kind<"PortsLayer">;
export type AdaptersLayer = Kind<"AdaptersLayer">;

export type AppContext = Kind<"AppContext", {
  ports: PortsLayer;
  adapters: AdaptersLayer;
}, {
  mustImplement: [["ports", "adapters"]];
}>;

export const app = {
  ports: {},
  adapters: {},
} satisfies InstanceConfig<AppContext>;
`);

console.log("=== Violation: RepositoryPort has no adapter ===");
await ksc("check", demo3);

In [ ]:
// Fix: add an adapter that implements the port
writeFile(demo3, "src/adapters/repository.adapter.ts", `
import { RepositoryPort } from '../ports/repository.port';

export class InMemoryRepositoryAdapter implements RepositoryPort {
  private store: unknown[] = [];

  save(entity: unknown): void {
    this.store.push(entity);
  }

  findAll(): unknown[] {
    return [...this.store];
  }
}
`);

console.log("=== Fixed: InMemoryRepositoryAdapter implements RepositoryPort ===");
await ksc("check", demo3);

cleanup(demo3);

---

## D. `noCycles` — No Circular Dependencies (KS70004)

Detects circular dependency chains between layers.

```typescript
noCycles: ["domain", "infrastructure"]
```

In [ ]:
const demo4 = copyFixture("no-cycles");

writeFile(demo4, "src/context.k.ts", `
import type { Kind, ConstraintConfig, InstanceConfig } from 'kindscript';

export type DomainLayer = Kind<"DomainLayer">;
export type InfrastructureLayer = Kind<"InfrastructureLayer">;

export type AppContext = Kind<"AppContext", {
  domain: DomainLayer;
  infrastructure: InfrastructureLayer;
}, {
  noCycles: ["domain", "infrastructure"];
}>;

export const app = {
  domain: {},
  infrastructure: {},
} satisfies InstanceConfig<AppContext>;
`);

console.log("=== Violation: domain <-> infrastructure cycle ===");
await ksc("check", demo4);

In [ ]:
// Fix: break the cycle — use dependency inversion
writeFile(demo4, "src/domain/service.ts", `
export interface DataStore {
  query(sql: string): string[];
}

export class DomainService {
  constructor(private store: DataStore) {}
  getData(): string[] { return this.store.query('SELECT *'); }
}
`);

writeFile(demo4, "src/infrastructure/database.ts", `
import { DataStore } from '../domain/service';

export class Database implements DataStore {
  query(sql: string): string[] { return [sql]; }
}
`);

console.log("=== Fixed: one-directional dependency (infra -> domain) ===");
await ksc("check", demo4);

cleanup(demo4);

---

## E. `filesystem.mirrors` — Every File Needs a Counterpart (KS70005)

Ensures files in one directory have matching files (by relative path) in another.

**Use case:** "Every component has a test file."

```typescript
filesystem: {
  mirrors: [["components", "tests"]]
}
```

In [ ]:
const demo5 = copyFixture("mirrors");

writeFile(demo5, "src/context.k.ts", `
import type { Kind, ConstraintConfig, InstanceConfig } from 'kindscript';

export type ComponentsLayer = Kind<"ComponentsLayer">;
export type TestsLayer = Kind<"TestsLayer">;

export type AppContext = Kind<"AppContext", {
  components: ComponentsLayer;
  tests: TestsLayer;
}, {
  filesystem: {
    mirrors: [["components", "tests"]];
  };
}>;

export const app = {
  components: {},
  tests: {},
} satisfies InstanceConfig<AppContext>;
`);

console.log("=== Violation: form.ts has no counterpart test ===");
await ksc("check", demo5);

In [ ]:
// Fix: add the missing test
writeFile(demo5, "src/tests/form.ts", `
import { Form } from '../components/form';
console.assert(Form() === 'form');
`);

console.log("=== Fixed: form.ts now has a test ===");
await ksc("check", demo5);

cleanup(demo5);

---

## F. `filesystem.exists` — Member Directories Must Exist (KS70010)

Checks that the derived directory paths for listed members actually exist on disk. This is **opt-in** — only members listed in `filesystem.exists` are checked.

**Use case:** You define structure in a `.k.ts` file and want to ensure the directories are created.

```typescript
filesystem: {
  exists: ["domain", "infrastructure"]
}
```

In [ ]:
const demo6 = copyFixture("existence");

writeFile(demo6, "src/context.k.ts", `
import type { Kind, ConstraintConfig, InstanceConfig } from 'kindscript';

export type DomainLayer = Kind<"DomainLayer">;
export type InfrastructureLayer = Kind<"InfrastructureLayer">;

export type AppContext = Kind<"AppContext", {
  domain: DomainLayer;
  infrastructure: InfrastructureLayer;
}, {
  filesystem: {
    exists: ["domain", "infrastructure"];
  };
}>;

export const app = {
  domain: {},
  infrastructure: {},
} satisfies InstanceConfig<AppContext>;
`);

console.log("=== Violation: infrastructure directory doesn't exist ===");
await ksc("check", demo6);

In [ ]:
// Fix: create the missing directory
Deno.mkdirSync(`${demo6}/src/infrastructure`, { recursive: true });
writeFile(demo6, "src/infrastructure/index.ts", `
// Infrastructure adapter placeholder
`);

console.log("=== Fixed: infrastructure directory created ===");
await ksc("check", demo6);

cleanup(demo6);

Without the `filesystem: { exists: [...] }` constraint, missing directories are silently ignored — dependency contracts like `noDependency` simply pass when a directory is empty or missing. Add `filesystem.exists` when you want to guarantee the directory structure matches your architectural definition.

---

## Summary

| Contract | Code | Catches | Fix pattern |
|----------|------|---------|-------------|
| `noDependency` | KS70001 | Forbidden imports between layers | Dependency injection — define interfaces in the inner layer |
| `mustImplement` | KS70002 | Port interfaces without adapters | Add a class that `implements` the interface |
| `purity` | KS70003 | I/O imports (`fs`, `http`, etc.) in pure layers | Inject I/O through constructor |
| `noCycles` | KS70004 | Circular dependency chains | Break the cycle with interfaces (Dependency Inversion) |
| `filesystem.mirrors` | KS70005 | Missing counterpart files | Add the missing file |
| `filesystem.exists` | KS70010 | Missing member directories | Create the missing directory |

Dependency contracts and purity are declared directly on the Kind type's third parameter. Filesystem constraints are grouped under a `filesystem` key:

```typescript
type DomainLayer = Kind<"DomainLayer", {}, { pure: true }>;

type CleanContext = Kind<"CleanContext", {
  domain: DomainLayer;
  infrastructure: InfrastructureLayer;
}, {
  noDependency: [["domain", "infrastructure"], ["domain", "application"]];
  mustImplement: [["application", "infrastructure"]];
  filesystem: {
    exists: ["domain", "infrastructure"];
    mirrors: [["components", "tests"]];
  };
}>;
```

The Kind type is the single source of truth for all architectural rules. If you need different constraints, define a different Kind.